In [ ]:
import os, sys
import vcgencmd as vc
import RPi.GPIO as gpio
from bottle   import Bottle, route, run, template
from datetime import datetime
import time

In [ ]:
# see https://pypi.org/project/vcgencmd/ for usage

In [ ]:
dirname = os.path.dirname(sys.argv[0])
dirname

In [ ]:
sys.argv

In [ ]:
dd = os.popen('pwd')
#dirname = os.path.dirname(dd.read())
pwd = dd.read()[:-1]
pwd

In [ ]:
vccmd = vc.Vcgencmd()

In [ ]:
vccmd.version()

In [ ]:
def get_temp():
    temp_C = vccmd.measure_temp()
    temp_F = temp_C * 9./5. +32.0
    return ["CPU TEMP:", "{:0.2f} deg C, {:0.2f} deg F".\
            format(temp_C, temp_F)]

def get_time():
    return ["TIME:", "{:%Y-%m-%d %H:%M:%S}".format(datetime.now())]

def get_load_average():
    one, five, fifteen = os.getloadavg()
    return ["PROCESS AVERAGES:", "{} (1m), {} (5m), {} (15m)".\
            format(one, five, fifteen)]

def get_uptime():
    dd = os.popen('/usr/bin/uptime -p')
    return ["UPTIME:", "{}".format(dd.read()[3:-1])]

def get_freq(obj='arm'):
    # obj = arm, core
    return ["FREQ of '{}':".format(obj), 
            "{:0.3f} GHz".format(vccmd.measure_clock(obj)/1.e9)]

In [ ]:
'''
def get_processes(num=5): # this version returns a text block
    # see unix.stackexchange.com #13968 : sorting on cpu%
    # top result is the header, so get n + 1
    cmd = '/bin/ps aux --sort=-pcpu | head -n {}'.format(num + 1)
    dd  = os.popen(cmd)
    txt = dd.read()
    out = [k.split() for k in txt.split('\n')]
    fields = [0, 1, 2, 3, 8, 9, 10] # limit the output fields
    txt = ''
    for k in fields:
        txt += out[0][k] + '\t'
    for j in range(1, len(out) - 1):
        txt += '\n'
        for k in fields: # split affects only the last field
            txt += out[j][k].split('/')[-1] + '\t'
    return txt
'''
def get_processes(num=5): # this version returns a list of lists
    # see unix.stackexchange.com #13968 : sorting on cpu%
    # top result is the header, so get n + 1
    cmd = '/bin/ps aux --sort=-pcpu | head -n {}'.format(num + 1)
    dd  = os.popen(cmd)
    txt = dd.read()
    # [:-1] gets rid of empty list after last \n
    out = [k.split() for k in txt.split('\n')][:-1]
    fields = [0, 1, 2, 3, 8, 9, 10] # limit the output fields
    short = []
    for j in range(len(out)): # split affects only the last field
        short.append([out[j][k].split('/')[-1] for k in fields])
    return short

In [ ]:
fns = [get_time, get_temp, get_load_average, get_uptime, get_freq]
dd1 = [k() for k in fns]
#dd1 = [k.split(':') for k in dd1]
for k in dd1:
    print(k)

In [ ]:
dd2 = get_processes(4)
dd2

In [ ]:
vccmd.get_sources?

In [ ]:
#get_sources("mem") will return ["arm", "gpu"] which are the sources/arguments for 
#get_mem(source) that returns the memory of the source passed as argument to it.
# the below dict from /usr/local/lib/python3.7/dist-packages/vcgencmd/vcgencmd.py
sources = {
  "clock": ["arm", "core","isp", "v3d", "uart", "pwm", "emmc", "pixel", 
            "vec", "hdmi", "dpi"], # deleted H264: gives error
  "volts": ["core", "sdram_c", "sdram_i", "sdram_p"],
  "mem":   ["arm", "gpu"],
  "codec": ["agif", "flac", "h263", "h264", "mjpa", "mjpb", "mjpg", "mpg2", 
            "mpg4", "mvc0", "pcm", "thra", "vorb", "vp6", "vp8", "wmv9", "wvc1"],
  "display_id": [0, 1, 2, 3, 7]
  }

In [ ]:
dd = vccmd.get_sources('mem')
for k in dd:
    print(k, vccmd.get_mem(k))

In [ ]:
dd = vccmd.get_sources('clock')
for k in dd:
    # H264 produces an error
    if (k != 'H264'): print(k, vccmd.measure_clock(k))

In [ ]:
dd = vccmd.get_sources('volts')
for k in dd:
    print(k, vccmd.measure_volts(k))

In [ ]:
dd = vccmd.get_sources('codec')
for k in dd:
    print(k, vccmd.codec_enabled(k))

In [ ]:
dd = vccmd.get_sources('display_id')
for k in dd:
    print(k, vccmd.display_power_state(k))

In [ ]:
vccmd.get_lcd_info()

In [ ]:
vccmd.dispmanx_list()

In [ ]:
#vccmd.otp_dump()

In [ ]:
#dev = os.popen('bin/ps -a', 'r', 1)
dev = os.popen('/opt/vc/bin/vcgencmd measure_temp')  # this works
dev.read()
#dev.close()

In [ ]:
os.getlogin()

In [ ]:
os.listdir()

In [ ]:
#os.sysconf_names # prints out a lot

# now play with bottle

In [ ]:
def index(name='RPi Status'):
    dt = get_time()
    tc = get_temp()
    info = {'name': name, 'dt': dt, 'tc': tc}
    return template('test.tpl', info)

In [ ]:
tt = index()
tt

In [ ]:
import sys
sys.path

In [ ]:
app = Bottle()

# MISC TESTING

In [ ]:
from gpiozero import LED, Button

In [ ]:
#Button?  # be careful using this: directly shorting gpio pins to ground

In [ ]:
button_names = ['RED', 'GREEN', 'BLUE', 'CYAN', 'MAGENTA', 'YELLOW',
                'WHITE', 'BLACK']
def html_for_led(led_number):
    i = str(led_number)
    result = " <input type='button'" +\
             " onClick='changed(" + i + ")'" +\
             " value='" + button_names[led_number] + "'/>"
             #" value='LED " + i + "'/>"
    return result

In [ ]:
html_for_led(7)

In [ ]:
leds = [LED(18), LED(23), LED(24)] # red, green, blue

In [ ]:
dd = [leds[0].on(), leds[1].on(), leds[2].on()]
for k in dd: k

In [ ]:
ee = [(1,0,0), (0,1,0), (0,0,1), (0,1,1), (1,0,1), (1,1,0),
      (1,1,1), (0,0,0)]

In [ ]:
for j in range(8):
    for i, k in enumerate(ee[j]):
        leds[i].on() if k==1 else leds[i].off()
    time.sleep(1)